In [18]:
from IPython.display import Image
import ee
ee.Initialize()
import sys 
sys.path.append('/rheil/')
import pandas as pd
import numpy as np
import simulation
import importlib
import gee_random
import gee_tools
importlib.reload(gee_random)
importlib.reload(simulation)
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns


In [19]:
rough_coords = [[[-72.70751953125,-35.55407805098183], 
                [-72.88330078125,-36.24230463768247],
                [-73.828125,-37.1758808189678],
                [-73.6083984375,-39.502157400405835],
                [-74.06982421875,-40.91166803248206],
                [-74.2236328125,-42.031161371349604],
                [-74.63012703694403,-43.39263031065478],
                [-71.53747566975653,-44.14235994137627],
                [-71.6748046875,-42.191447917682844],
                [-71.7681884765625,-40.913051951779906],
                [-70.7904052734375,-38.63355979178271],
                [-71.0540771484375,-36.817591733270724],
                [-70.6640625,-36.43405098759998],
                [-70.279541015625,-35.96862121673053],
                [-70.4058837890625,-35.24960684089411],
                [-69.78515616618097,-34.21584009811081],
                [-69.73571768961847,-33.3529636230323],
                [-69.87304683309048,-33.095632114465566],
                [-70.15319820027798,-32.28661649357072],
                [-70.81237788777798,-31.960965572989558],
                [-71.58142085652798,-32.147194134857614],
                [-72.70751953125,-35.55407805098183]]]
bounds = ee.Geometry.Polygon(rough_coords)


# chile_comunas = ee.FeatureCollection('ft:1MIVtGXFEu3ONr87OPP7JV1_vCs9W3sJEuZDuzkzb')
# chile_comunas = chile_comunas.filter(ee.Filter.inList('id_1', [15, 13, 8, 12, 6, 3, 10, 9]))
# def simplify(feature):
#     feature = feature.simplify(1)
# chile_comunas = chile_comunas.map(simplify, True)


# chile = ee.Feature(ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')\
#             .filterMetadata('Country', 'equals', 'Chile').first()).geometry().getInfo()

# # regions = ee.FeatureCollection('ft:1RfoT4UoTohqmJJrNkMT5Jyp95SahGk5f9V355SnH')
# # regions = regions.filter(ee.Filter.inList('COD_REG', ['05', '06', '07', '08', '09', '10', '13', '14']))
# # reg = regions.filterMetadata('COD_REG', 'equals', '08')
coefs_csv = '/rheil/chile/data/pooled_coefs.csv'
coefs_se_csv = '/rheil/chile/data/pooled_se.csv'
asset_dir = 'users/rheilmayr/chile_sim/input/'
out_dir = 'users/rheilmayr/chile_sim/simulation/'

sample_region = ee.Algorithms.GeometryConstructors.Polygon([[-73.25,-37.4], [-73.25,-37.6], 
    [-73,-37.6], [-73,-37.4], [-73.25,-37.4]])
sample_region = ee.Feature(sample_region)

np.random.seed(1235)

In [20]:
n = 1
scn_key = 'rs'
scale = 30
reg_id = '08'

In [21]:
base_dict ={'agrent': asset_dir + 'agrent_01',
            'forrent': asset_dir + 'forrent',
            'south': asset_dir + 'south',
            'central': asset_dir + 'central',
            'noprot': asset_dir + 'noprot',
            'luq2': asset_dir + 'luq2',
            'luq3': asset_dir + 'luq3',
            'mask': asset_dir + 'mask'}

map_dicts = {key: base_dict.copy() for key in ['01_sub', '01_ns', '11_rs', '11_ns', '11_sub']}
map_dicts['01_sub'].update({'plantrent': asset_dir + 'plantrent_01',
                            'olu': asset_dir + 'olu_1986'})
map_dicts['01_ns'].update({'plantrent': asset_dir + 'plantrent_01_ns',
                           'olu': asset_dir + 'olu_1986'})
map_dicts['11_rs'].update({'plantrent': asset_dir + 'plantrent_11_rs'})
map_dicts['11_ns'].update({'plantrent': asset_dir + 'plantrent_11_ns'})
map_dicts['11_sub'].update({'plantrent': asset_dir + 'plantrent_11'})

scn_dict = {'sub': ['01_sub', '11_sub'],
            'ns': ['01_ns', '11_ns'],
            'rs': ['01_sub', '11_rs']}

Run simulation

In [22]:
# Load period 1 input ic
input_ic_p1 = simulation.dict_to_input_ic(map_dicts[scn_dict[scn_key][0]], n)

# Run period 1 simulation
#pr_ic_p1, cum_pr_ic_p1, clas_ic_p1 = simulation.sim_flow(input_ic_p1, coefs_csv, coefs_se_csv, n)
pr_ic_p1, cum_pr_ic_p1, clas_ic_p1 = simulation.sim_flow_norand(input_ic_p1, coefs_csv)

# Load period 2 input ic, add period 1 simulated map
input_ic_p2 = simulation.dict_to_input_ic(map_dicts[scn_dict[scn_key][1]], n)
rename_olu = gee_tools.band_renamer(['clas'], ['olu'])
clas_ic_p1 = clas_ic_p1.map(rename_olu)
input_ic_p2 = input_ic_p2.combine(clas_ic_p1)

# Run period 2 simulation
pr_ic_p2, cum_pr_ic_p2, clas_ic_p2 = simulation.sim_flow(input_ic_p2, coefs_csv, coefs_se_csv, n)

Export classifications to assets

In [15]:
exports = []
for i in range(0,n):
    img = clas_ic_p2.toList(count = 1, offset = i).get(0)
    exports.append(ee.batch.Export.image.toAsset(image = img, description = 'clas_' + scn_key + str(i), 
                                           assetId = 'users/rheilmayr/chile_sim/sim_ic/clas_' + scn_key + '_' + str(i),
                                           scale = scale, maxPixels = 1e11, region = bounds))


    

In [16]:
i = 0
img = clas_ic_p2.toList(count = 1, offset = i).get(0)
ee.batch.Export.image.toAsset(image = img, description = 'clas_' + scn_key + str(i), 
                              assetId = 'users/rheilmayr/chile_sim/sim_ic/clas_' + scn_key + '_' + str(i),
                              scale = scale, maxPixels = 1e11, region = bounds).start()


In [17]:
for export in exports[1:]:
    export.start()

Tabulate results

In [23]:
regions = ee.Image(asset_dir + 'regions')
class add_region_map:
    def __init__(self, regions):
        self.regions = regions
    def __call__(self, img):
        img = regions.multiply(100).add(img)
        return img
add_region_mapper = add_region_map(regions)    
clas_ic_p2_reg = clas_ic_p2.map(add_region_mapper)
#clas_ic_p2_reg.getInfo()

In [24]:
regions = [5, 6, 7, 8, 9, 10, 13, 14]
lu_types = [1, 3, 5, 19]
import itertools
values = []
for r in itertools.product(regions, lu_types):
    values.append(r[0] * 100 + r[1])
# img = ee.Image(clas_ic_p2_reg.first())

class sum_clas_map:
    def __init__(self, values, scale):
        self.values = values
        self.old_bands = ['constant']
        self.old_bands.extend(['constant_' + str(i+1) for i in range(len(values)-1)])
        self.new_bands = ['cr_' + str(v) for v in values]
        self.scale = scale
    def __call__(self, img):
#        img = img.clip(sample_region)
        areas_img = img.eq(self.values).multiply(ee.Image.pixelArea().divide(1e7))
        areas_img = areas_img.select(self.old_bands, self.new_bands)
#        areas_dict = areas_img.reduceRegion(reducer = ee.Reducer.sum(), scale = self.scale, 
#                                            maxPixels = 1e11)
        areas_dict = areas_img.reduceRegion(reducer = ee.Reducer.sum(), scale = self.scale, 
                                            maxPixels = 1e11, geometry = bounds)
        out_point = ee.Feature(ee.Geometry.Point(0, 0))
        out_point = out_point.set(areas_dict)
        return out_point


In [25]:
sum_clas_mapper = sum_clas_map(values, scale)
clas_sum_fc = clas_ic_p2_reg.map(sum_clas_mapper)

In [26]:
task = ee.batch.Export.table.toDrive(collection = clas_sum_fc, 
                                     description = 'lusim_n' + str(n) + '_s' + str(scale) + '_' + scn_key, 
                                     folder = 'chile/sim/', fileFormat = 'csv')
task.start()

Single region tabulation

In [ ]:
class reg_clip_map:
    def __init__(self, region):
        self.region = region
    def __call__(self, img):
        img = img.clip(self.region)
        return img

values = [1, 3, 5, 19]
regions = ee.FeatureCollection('ft:1RfoT4UoTohqmJJrNkMT5Jyp95SahGk5f9V355SnH')
regions = regions.filter(ee.Filter.inList('COD_REG', ['05', '06', '07', '08', '09', '10', '13', '14']))
reg = regions.filterMetadata('COD_REG', 'equals', reg_id)
reg_clip_mapper = reg_clip_map(reg)
clas_ic_p2_reg = clas_ic_p2.map(reg_clip_mapper)
sum_clas_mapper = sum_clas_map(values, scale)
clas_sum_fc = clas_ic_p2_reg.map(sum_clas_mapper)    
task = ee.batch.Export.table.toDrive(collection = clas_sum_fc, 
                                     description = 'lusim_n' + str(n) + '_s' + str(scale) + '_r' + reg_id + '_' + scn_key, 
                                     folder = 'chile/sim/', fileFormat = 'csv')
task.start()

Tabulate results (region by region)

In [7]:
region_bounds = {5: [[[-72.13348388671875,-34.143271868329485], 
                      [-69.89501953125,-33.13681647430008],
                      [-70.12023917399347,-31.87214800755421],
                      [-71.79565421305597,-32.142317351767744],
                      [-72.13348388671875,-34.143271868329485]]],
                 13: [[[-72.22412109375,-34.060307027573366],
                       [-69.697265625,-34.36919591276987],
                       [-69.63134765625,-32.859658269759734],
                       [-72.1142578125,-32.859658269759734],
                       [-72.22412109375,-34.060307027573366]]],
#                  6: [[]],
#                  7: [[]],
#                  8: [[]],
#                  9: [[]],
                 14: [[[-73.916015625,-40.35866682438446],
                       [-71.65283203125,-40.91723140476455],
                       [-71.3671875,-39.49088288923408],
                       [-73.6138916015625,-39.16795653371165],
                       [-73.916015625,-40.35866682438446]]],
                 10: [[[-73.89404296875,-40.05950548053006],
                       [-74.6630859375,-43.33949606998904],
                       [-71.56494107097387,-44.337928878298605],
                       [-71.4495849609375,-40.661672432741554],
                       [-73.89404296875,-40.05950548053006]]]}
region_bounds = {key: ee.Geometry.Polygon(bounds).toGeoJSON() for key, bounds in region_bounds.items()}
# bounds = ee.Geometry.Polygon(rough_coords).toGeoJSON()['coordinates']
# region_bounds

In [8]:
region_img = ee.Image(asset_dir + 'regions')
class add_region_map:
    def __init__(self, region_img):
        self.region_img = region_img
    def __call__(self, img):
        img = self.region_img.multiply(100).add(img)
        return img
add_region_mapper = add_region_map(region_img)    
clas_ic_p2_reg = clas_ic_p2.map(add_region_mapper)

In [13]:
regions = [5]
region = regions[0]
lu_types = [1, 3, 5, 19]
import itertools
values = []
for r in itertools.product(regions, lu_types):
    values.append(r[0] * 100 + r[1])
# img = ee.Image(clas_ic_p2_reg.first())

masker = gee_tools.mask_update_map(region_img.eq(region))
tab_ic = clas_ic_p2_reg.map(masker)

class sum_clas_map:
    def __init__(self, values, scale, bounds):
        self.values = values
        self.old_bands = ['constant']
        self.old_bands.extend(['constant_' + str(i+1) for i in range(len(values)-1)])
        self.new_bands = ['cr_' + str(v) for v in values]
        self.scale = scale
        self.bounds = bounds
    def __call__(self, img):
#         img = img.clip(sample_region)
        areas_img = img.eq(self.values).multiply(ee.Image.pixelArea().divide(1e7))
        areas_img = areas_img.select(self.old_bands, self.new_bands)
        areas_dict = areas_img.reduceRegion(reducer = ee.Reducer.sum(), scale = self.scale, 
                                            maxPixels = 1e11, geometry = self.bounds)
        out_point = ee.Feature(ee.Geometry.Point(0, 0))
        out_point = out_point.set(areas_dict)
        return out_point


In [14]:
sum_clas_mapper = sum_clas_map(values, scale, region_bounds[region])
clas_sum_fc = clas_ic_p2_reg.map(sum_clas_mapper)

In [15]:
task = ee.batch.Export.table.toDrive(collection = clas_sum_fc, 
                                     description = 'lusim_n' + str(n) + '_s' + str(scale) + \
                                     '_' + scn_key +'_r' +str(region), 
                                     folder = 'chile/sim/', fileFormat = 'csv')
task.start()

Deprecated code?

In [ ]:
clas_map = ee.Image(asset_dir + 'olu_1986')
regions = ee.Image(asset_dir + 'regions')
add_region_mapper = add_region_map(regions)    
clas_map = add_region_mapper(clas_map)
scale = 30
sum_clas_mapper = sum_clas_map(values, scale)
clas_sum_fc = ee.FeatureCollection(sum_clas_mapper(clas_map))

In [ ]:
task = ee.batch.Export.table.toDrive(collection = clas_sum_fc, 
                                     description = 'olu_1986' + '_s' + str(scale), 
                                     folder = 'chile/sim/', fileFormat = 'csv')
task.start()

In [ ]:
def sum_clas_area(img, clas, fc, scale = 30, n = 1):
    """
    Parameters
    ----------
    img: ee.Image
        Classified image
       
    clas: int
        Class to calculate area (thousand has) over
    
    fc: ee.FeatureCollection
        Features to sum within
        
    scale: scale
        ee scale parameter
        
    n: int
        Number of classified bands included in img. Allows for
        comparison of areas resulting from different classified maps
       
    Returns
    ------
    fc: ee.FeatureCollection
        fc, but with new properties summing the area of the class
        within each feature, for each classified band of the img
    """
    binary_img = img.eq(clas)
    clasarea_img = binary_img.multiply(ee.Image.pixelArea().divide(1e7))
    new_bands = ['c' + str(clas) + '_' + str(i) for i in range(n)]
    clasarea_img = clasarea_img.select(img_bands, new_bands)
    fc = clasarea_img.reduceRegions(collection = fc, scale = scale, reducer = ee.Reducer.sum())
    return fc


clas_img = clas_ic_p2.iterate(gee_tools.band_aggregator, ee.Image())
img_bands = ['clas_' + str(i+1) for i in range(n-1)]
img_bands.extend(['clas'])
clas_img = ee.Image(clas_img).select(img_bands)
for c in [1, 3, 5, 19]:
    chile_comunas = sum_clas_area(clas_img, c, chile_comunas, 500, n)

In [ ]:
task = ee.batch.Export.table.toDrive(collection = chile_comunas, description = 'lusim_n' + str(n) + '_' + scn_key, 
                                     folder = 'chile/sim/', fileFormat = 'csv')
task.start()

In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p2.first()).clip(sample_region), description = 'cum_pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p2.first()).clip(sample_region), description = 'pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p2.first()).clip(sample_region), description = 'clas_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()


In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_img), description = 'clas_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas_img_ns', 
                                     scale = 500, maxPixels = 1e10)
task.start()


In [ ]:
time_period = 'pooled'
n = 3
coefs_df = simulation.load_coefs(coefs_csv, time_period)
coefs_se_df = simulation.load_coefs(coefs_se_csv, time_period)

x = draws.pop()
len(draws)

In [ ]:
results_dir = '/rheil/notebooks/data/gee_results/'
results_dict  = {key: results_dir + 'lusim_n6_' + key + '.csv' for key in ['sub', 'ns', 'rs']}
results_df_dict = {}
for key, result in results_dict.items():
    results_df_dict[key] = pd.read_csv(result)
results_df = pd.concat(results_df_dict)
results_df = results_df.set_index('COD_REG', append = True)
keep_cols = [col for col in results_df.columns if col[0] == 'c']
results_df = results_df[keep_cols]
clas = [col[1:col.rfind('_')] for col in results_df.columns]
run = [col[col.rfind('_')+1:] for col in results_df.columns]
results_df = results_df.T
results_df['clas'] = clas
results_df['run'] = run
results_df = results_df.set_index(['clas', 'run'])

In [ ]:
results_df.sum(axis = 1, level = 0).loc['1']['ns']

In [ ]:
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['ns'], results_df.sum(axis = 1, level = 0).loc['3']['ns'],
                 cmap="Reds", shade=True, shade_lowest=False)
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['sub'], results_df.sum(axis = 1, level = 0).loc['3']['sub'],
                 cmap="Greens", shade=True, shade_lowest=False)
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['rs'], results_df.sum(axis = 1, level = 0).loc['3']['rs'],
                 cmap="Blues", shade=True, shade_lowest=False)

In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p2.first()).clip(sample_region), description = 'cum_pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p2.first()).clip(sample_region), description = 'pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p2.first()).clip(sample_region), description = 'clas_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p1.first()).clip(sample_region), description = 'cum_pr_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p1.first()).clip(sample_region), description = 'pr_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/pr_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p1.first()).clip(sample_region), description = 'clas_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/clas_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()

In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p2.first()), description = 'cum_pr_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p2.first()), description = 'pr_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/pr', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p2.first()), description = 'clas_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas', 
                                     scale = 30, maxPixels = 1e10)
task.start()

In [ ]:
# task = ee.batch.Export.image.toAsset(image = ee.Image(input_ic_p2.first()).clip(sample_region), description = 'input_p2', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/inputs_p2', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(input_ic_p1.first()).clip(sample_region), description = 'input_p1', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/inputs_p1', 
                                     scale = 30, maxPixels = 1e10)
task.start()

In [ ]:
results_df.sum(axis = 1)

In [ ]:
luq2 = ee.Image(asset_dir + 'luq2')
luq3 = ee.Image(asset_dir + 'luq3')
noprot = ee.Image(asset_dir + 'noprot')
central = ee.Image(asset_dir + 'central')
south = ee.Image(asset_dir + 'south')
agrent01 = ee.Image(asset_dir + 'agrent_01')
agrent11 = ee.Image(asset_dir + 'agrent_11')
plantrent01 = ee.Image(asset_dir + 'plantrent_01')
plantrent11 = ee.Image(asset_dir + 'plantrent_11')
forrent = ee.Image(asset_dir + 'forrent')
olu_1986 = ee.Image(asset_dir + 'olu_1986')
mask = (luq2.eq(0).Or(luq2.eq(1)))\
            .And(luq3.eq(0).Or(luq3.eq(1)))\
            .And(noprot.eq(0).Or(noprot.eq(1)))\
            .And(central.eq(0).Or(central.eq(1)))\
            .And(south.eq(0).Or(south.eq(1)))\
            .And(agrent01.gt(-3.4e38))\
            .And(agrent11.gt(-3.4e38))\
            .And(plantrent01.gt(-3.4e38))\
            .And(plantrent11.gt(-3.4e38))\
            .And(forrent.gt(-3.4e38))\
            .And(olu_1986.eq(1).Or(olu_1986.eq(3)).Or(olu_1986.eq(5)).Or(olu_1986.eq(19)))
        
task = ee.batch.Export.image.toAsset(image = mask, description = 'mask', 
                                     assetId = 'users/rheilmayr/chile_sim/inputs/mask_new', 
                                     scale = 30, maxPixels = 1e10)
task.start()


In [ ]:
# var multiMax = function(feature) {
#   // Reduce to mean value the input polygon for each image
#   var means = collection.map(function(image) {
#       var a = image.reduceRegion({
#         reducer: ee.Reducer.mean(),
#         geometry: feature.geometry(),
#         scale: 250
#       });
#       // Extract the reduced value from the returned dictionary and add it 
#       // to the image as property 'mean'.
#       image = image.set('mean', a.get('EVI'));
#       return image;
#   });

class multiMapAreas:
    def __init__(self, ic):
        self.ic = ic
    def sum_features(self, feature):
        self.feature = feature
        areas_ic = self.ic.map(self.sum_img)
        return areas_ic
    def sum_img(self, img):
        areas = img.reduceRegion(reducer = ee.Reducer.sum(),
                                 geometry = self.feature.geometry(),
                                 scale = 500)
        clases = ['for', 'plant', 'shrub', 'ag']
        clas_dict = {}
        for clas in clases:
            clas_dict[clas] = areas.get(clas)
        img = img.set(clas_dict)
        return img
    
# def sum_ic(feature):
#     def sum_img(img):
#         areas = img.reduceRegion(reducer = ee.Reducer.sum(),
#                                  geometry = feature.geometry(),
#                                  scale = 500)
#         return areas
#     areas = clas_ic.map(sum_img)
#     clases = ['for', 'plant', 'shrub', 'ag']
#     clas_dict = {}
#     for clas in clases:
#         clas_dict[clas] = areas.get(clas)
#     img = img.set(clas_dict)
#     return img

# clas_ic = clas_ic.map(sum_freqs)
# clas_info = clas_ic.getInfo()

In [ ]:
multiMapper = multiMapAreas(clas_ic)
areas_ic = regions.map(multiMapper.sum_features)
areas_ic.getInfo()

In [ ]:
img = ee.Image(clas_ic.first())
feature = ee.Feature(reg.first())

def sum_img(img):
    areas = img.reduceRegion(reducer = ee.Reducer.sum(),
                             geometry = feature.geometry(),
                             scale = 500)
    clases = ['for', 'plant', 'shrub', 'ag']
    clas_dict = {}
    img_id = img.get('system:index')
    for clas in clases:
        clas_dict['r' + img_id + '_' + clas] = areas.get(clas)
    img = img.set(clas_dict)
    return img
sum_img(img).getInfo()

In [ ]:
clas_img = ee.Image(clas_ic.first())
area_img = clas_img.multiply(ee.Image.pixelArea().divide(10000000))
def add_areas(feature):
    clases = ['for', 'plant', 'shrub', 'ag']
    areas = area_img.reduceRegion(reducer = ee.Reducer.sum(), 
                                    geometry = feature.geometry(), 
                                    scale = 1000, 
                                    maxPixels = 1e10)
    clas_dict = {}
    for clas in clases:
        clas_dict[clas] = areas.get(clas)
    feature = feature.set(clas_dict)
    return feature

out_regions = regions.map(add_areas)

In [ ]:
out_dict = {}
for i, region_dict in enumerate(test_info['features']):
    out_dict[i] = region_dict['properties']
out_df = pd.DataFrame(out_dict)
out_df = out_df.T
out_df = out_df.set_index('COD_REG').sort_index()
out_df[['for', 'plant', 'shrub', 'ag']]

In [ ]:
stats = clas_img.reduceRegion(reducer = ee.Reducer.frequencyHistogram(), 
                                      geometry = reg, 
                                      scale = 100, 
                                      maxPixels = 9e9)
stats.getInfo()

In [ ]:
Image(url=clas_img.select('for').getThumbUrl({'min':0, 'max':1}))

In [ ]:
#mask = map_dict['olu'].eq(0).Or(map_dict['olu'].eq(1))
#pr_img = pr_img.updateMask(mask)
pr_dict = {}
pr_dict['ag'] = pr_img.select(['ag'], ['prob']).addBands(ee.Image.constant(19).byte())
pr_dict['shrub'] = pr_img.select(['shrub'], ['prob']).addBands(ee.Image.constant(5).byte())
pr_dict['for'] = pr_img.select(['for'], ['prob']).addBands(ee.Image.constant(1).byte())
pr_dict['plant'] = pr_img.select(['plant'], ['prob']).addBands(ee.Image.constant(3).byte())
pr_collection = ee.ImageCollection([pr_dict['for'], pr_dict['plant'], pr_dict['shrub'], pr_dict['ag']])
mosaic = pr_collection.qualityMosaic('prob').select(['constant'], ['elu'])
transition_img = map_dict['olu'].multiply(100).add(mosaic)

transitions = [101, 103, 105, 119, 301, 303, 305, 319, 501, 
               503, 505, 519, 1901, 1903, 1905, 1919]
new_bands = ['constant']
additional_bands = ['constant_' + str(i) for i in np.arange(1,16)]
new_bands.extend(additional_bands)
t_bands = [str(i) for i in transitions]
transition_bands = transition_img.eq(transitions).multiply(ee.Image.pixelArea().divide(10000000))
transition_bands = transition_bands.select(new_bands, t_bands)


def add_change_areas(feature):
    changes = transition_bands.reduceRegion(reducer = ee.Reducer.sum(), 
                                            geometry = feature.geometry(), 
                                            scale = 30, 
                                            maxPixels = 5e9)
    t_dict = {}
    for t in transitions:
        t_dict[str(t)] = changes.get(str(t))
        feature = feature.set(t_dict)
    return feature

out = regions.map(add_change_areas)
task = ee.batch.Export.table.toDrive(collection = out, description = 'Sim_test_' + str(n),  
                                     fileFormat = 'csv', fileNamePrefix = 'sim_test_' + str(n))
task.start()

In [ ]:
regions.getInfo()

In [ ]:
## Investigate regions

band = 'Ran_uniform'
# img = ee.Image(cum_pr_ic.first()).select(band)
runi =  gee_random.runiform_raster()
img = runi(img)
hist = img.reduceRegion(reducer = ee.Reducer.frequencyHistogram(), geometry = sample_region.geometry(), scale = 30)
hist_info = hist.getInfo()
series = pd.Series(hist_info[band])


In [ ]:
pr_img = ee.Image(asset_dir + 'plantrent_11_rs')
mask = pr_img.gt(-1e30)
pr_img = pr_img.updateMask(mask)
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_img), description = 'plantrent_rs_masked', 
                                     assetId = 'users/rheilmayr/chile_sim/input/plantrent_rs_masked', 
                                     scale = 30, maxPixels = 1e10)
task.start()


In [ ]:
coefs_df_all_p1